In [1]:
import os

In [2]:
%pwd

'd:\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Chicken-Disease-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [74]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import read_yaml, create_dir

In [75]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

                 self.config = read_yaml(config_filepath)
                 self.params = read_yaml(params_filepath)

                 create_dir([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:

            config = self.config.data_ingestion

            create_dir([config.root_dir])

            data_ingestion_config = DataIngestionConfig(

                    root_dir=config.root_dir,
                    source_URL=config.source_URL,
                    local_data_file=config.local_data_file,
                    unzip_dir=config.unzip_dir
            )

            return data_ingestion_config



In [84]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassification import logger
from ChickenDiseaseClassification.utils.common import get_size
import logging

In [85]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.logger = logging.getLogger()  # Obtain a logger instance

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file)

            self.logger.info(f"Downloaded {filename} with info: \n{headers}")
        else:
            self.logger.info(f"File {self.config.local_data_file} already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [88]:
try:
    config = ConfigurationManager(config_filepath=str(CONFIG_FILE_PATH), params_filepath=str(PARAMS_FILE_PATH))
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-23 19:03:01,306: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-23 19:03:01,308: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-23 19:03:01,394: INFO: common: Creating directory: artifacts]
[2024-03-23 19:03:01,397: INFO: common: Creating directory: artifacts/data_ingestion]
[2024-03-23 19:03:02,654: INFO: 92929981: Downloaded artifacts/data_ingestion/data.zip with info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1B80:300682:64CA72:71C71A:65FF5FA2
Accept-Ranges: bytes
Date: Sat, 23 Mar 2024 23:03:02 GMT
Via: 1.1 varnish
X-Served-By: cache-chi-klot8100075-CHI
X-Cache: MISS
X-C

In [81]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')